In [1]:
%pip install ipython-autotime
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=3f561a5fa11fb1a25e25c8099400801b4ddd406f26eec37ef37ff35c0d0ee9f9
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
%%script echo 'ignore cell'
import os
import sys

# resolver problema de versão python x pyspark workers
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

ignore cell


In [3]:
from google.colab          import drive, files
from pyspark.sql           import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types     import *

%load_ext autotime

time: 280 µs (started: 2023-09-14 16:34:55 +00:00)


In [4]:
drive.mount('/content/drive', force_remount=True)
spark = SparkSession.builder.master('local').appName('pyspark_app').getOrCreate()
spark

Mounted at /content/drive


time: 37.4 s (started: 2023-09-14 16:34:55 +00:00)


In [5]:
df = spark.read.csv("/content/drive/MyDrive/datasets/housing/housing.csv", header=True, inferSchema=True, encoding='utf-8')
print(type(df))
print(f'rows: {df.count()}')
print(f'cols: {len(df.columns)}')
df.show(5)

<class 'pyspark.sql.dataframe.DataFrame'>
rows: 20640
cols: 10
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|       

In [6]:
print(df.columns)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity']
time: 551 µs (started: 2023-09-14 16:35:48 +00:00)


In [7]:
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)

time: 14.5 ms (started: 2023-09-14 16:35:48 +00:00)


Verificação de valores NaN.

In [8]:
for column in df.columns:
  mask = df['longitude'].isNull()
  nan_amount = df.filter(mask).count()
  print(f'{column}: {nan_amount}')

longitude: 0
latitude: 0
housing_median_age: 0
total_rooms: 0
total_bedrooms: 0
population: 0
households: 0
median_income: 0
median_house_value: 0
ocean_proximity: 0
time: 7.01 s (started: 2023-09-14 16:35:48 +00:00)


Renomeação das colunas transformando tudo em letras maiúsculas.

In [9]:
upper = [column.upper() for column in df.columns]
for column, up in zip(df.columns, upper):
  df = df.withColumnRenamed(column, up)
print(df.columns)

['LONGITUDE', 'LATITUDE', 'HOUSING_MEDIAN_AGE', 'TOTAL_ROOMS', 'TOTAL_BEDROOMS', 'POPULATION', 'HOUSEHOLDS', 'MEDIAN_INCOME', 'MEDIAN_HOUSE_VALUE', 'OCEAN_PROXIMITY']
time: 137 ms (started: 2023-09-14 16:35:55 +00:00)


Renomeação das colunas transformando tudo em letras minúsculas.

In [10]:
lower = [column.lower() for column in df.columns]
for column, low in zip(df.columns, lower):
  df = df.withColumnRenamed(column, low)
print(df.columns)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity']
time: 146 ms (started: 2023-09-14 16:35:55 +00:00)


Selecionando colunas.

In [11]:
print(type(df.select(['longitude', 'latitude', 'households'])))
df.select(['longitude', 'latitude', 'households']).show(5)

<class 'pyspark.sql.dataframe.DataFrame'>
+---------+--------+----------+
|longitude|latitude|households|
+---------+--------+----------+
|  -122.23|   37.88|     126.0|
|  -122.22|   37.86|    1138.0|
|  -122.24|   37.85|     177.0|
|  -122.25|   37.85|     219.0|
|  -122.25|   37.85|     259.0|
+---------+--------+----------+
only showing top 5 rows

time: 391 ms (started: 2023-09-14 16:35:55 +00:00)


Selecionando colunas. Forma alternativa.

In [12]:
print(type(df.select([col('latitude'), col('longitude'), col('households')])))
df.select([col('latitude'), col('longitude'), col('households')]).show(5)

<class 'pyspark.sql.dataframe.DataFrame'>
+--------+---------+----------+
|latitude|longitude|households|
+--------+---------+----------+
|   37.88|  -122.23|     126.0|
|   37.86|  -122.22|    1138.0|
|   37.85|  -122.24|     177.0|
|   37.85|  -122.25|     219.0|
|   37.85|  -122.25|     259.0|
+--------+---------+----------+
only showing top 5 rows

time: 351 ms (started: 2023-09-14 16:35:56 +00:00)


Atribuindo alias para cada coluna selecionada. Só é possível atribuir alias à colunas através da função col() que retorna um objeto Column.


In [13]:
lat = col('latitude').alias('lat')
lon = col('longitude').alias('lon')

print(lat)
df.select([lat, lon]).show(5)

Column<'latitude AS lat'>
+-----+-------+
|  lat|    lon|
+-----+-------+
|37.88|-122.23|
|37.86|-122.22|
|37.85|-122.24|
|37.85|-122.25|
|37.85|-122.25|
+-----+-------+
only showing top 5 rows

time: 485 ms (started: 2023-09-14 16:35:56 +00:00)


Aplicação de filtros.

In [23]:
mask = (col('ocean_proximity') == 'NEAR BAY')
print(mask)
df.where(mask).show(5)

Column<'(ocean_proximity = NEAR BAY)'>
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR 

In [14]:
mask = (col('ocean_proximity') == 'NEAR BAY')
print(mask)
df.filter(mask).show(5)

Column<'(ocean_proximity = NEAR BAY)'>
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR 

Filtros compostos.

In [24]:
mask = (col('ocean_proximity') == 'NEAR BAY') & (col('median_income') < 5.0)
print(mask)
df.where(mask).show(5)

Column<'((ocean_proximity = NEAR BAY) AND (median_income < 5.0))'>
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.25|   37.85|              52.0|     1627.0|         280.0|     565.0|     259.0|       3.8462|          342200.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|      919.0|         213.0|     413.0|     193.0|       4.0368|          269700.0|       NEAR BAY|
|  -122.25|   37.84|              52.0|     2535.0|         489.0|    1094.0|     514.0|       3.6591|          299200.0|       NEAR BAY|
|  -122.25|   37.84|              52.0|     3104.0|         687.0|    1157.0|     647.0|         3.12|   

In [15]:
mask = (col('ocean_proximity') == 'NEAR BAY') & (col('median_income') < 5.0)
print(mask)
df.filter(mask).show(5)

Column<'((ocean_proximity = NEAR BAY) AND (median_income < 5.0))'>
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.25|   37.85|              52.0|     1627.0|         280.0|     565.0|     259.0|       3.8462|          342200.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|      919.0|         213.0|     413.0|     193.0|       4.0368|          269700.0|       NEAR BAY|
|  -122.25|   37.84|              52.0|     2535.0|         489.0|    1094.0|     514.0|       3.6591|          299200.0|       NEAR BAY|
|  -122.25|   37.84|              52.0|     3104.0|         687.0|    1157.0|     647.0|         3.12|   

Criar novas colunas.

In [16]:
# atribuo um valor literal(True) à nova coluna chamada 'new_col'
df.withColumn('new_col', lit(True)).show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|new_col|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|   true|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|   true|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|   true|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|   

In [17]:
result = col('total_bedrooms') / col('total_rooms')
df.withColumn('new_col', lit(result)).show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|            new_col|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|0.14659090909090908|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|0.15579659106916466|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|0.12951601908657123|
|  -122.25|   37.85|              52.0|     1274.0| 

In [26]:
# nova coluna usando a função substring()
df.withColumn('new_col', substring('ocean_proximity', 1, 4)).show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|new_col|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|   NEAR|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|   NEAR|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|   NEAR|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|   

In [29]:
# concatenando duas colunas para formar uma nova
df.withColumn('new_col', concat(col('latitude'), col('longitude'))).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|     new_col|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|37.88-122.23|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|37.86-122.22|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|37.85-122.24|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|      

In [32]:
# concatenando duas colunas para formar uma nova
df.withColumn('new_col', concat_ws(' # ', col('latitude'), col('longitude'))).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|        new_col|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|37.88 # -122.23|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|37.86 # -122.22|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|37.85 # -122.24|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0

Datas: Vou usar outro dataset que contenha datas em formato string para fazer a conversão de string para o formato YYY-MM-DD

In [39]:
df_ = spark.read.csv("/content/drive/MyDrive/datasets/wc2018-players.csv", header=True, inferSchema=True, encoding='utf-8')
print(type(df_))
print(f'rows: {df_.count()}')
print(f'cols: {len(df_.columns)}')
df_.show(5)

<class 'pyspark.sql.dataframe.DataFrame'>
rows: 736
cols: 9
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows

time: 991 ms (started: 2023-09-14 17:09:56

In [40]:
df_.printSchema()

root
 |-- Team: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- Pos.: string (nullable = true)
 |-- FIFA Popular Name: string (nullable = true)
 |-- Birth Date: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)

time: 6.77 ms (started: 2023-09-14 17:10:01 +00:00)


Note que 'Birth Date' está no formato string e também o separador dos componentes YYY, MM, DD é um ponto. Vamos usá-lo como referência.

In [65]:
dia = udf(lambda date:date.split('.')[0])
mes = udf(lambda date:date.split('.')[1])
ano = udf(lambda date:date.split('.')[2])

df_.withColumn('Dia', dia('Birth Date')).show(5)
df_.withColumn('Mes', mes('Birth Date')).show(5)
df_.withColumn('Ano', ano('Birth Date')).show(5)

+---------+---+----+------------------+----------+----------+--------------------+------+------+---+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|Dia|
+---------+---+----+------------------+----------+----------+--------------------+------+------+---+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65| 31|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65| 21|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66| 15|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69| 13|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72| 24|
+---------+---+----+------------------+----------+----------+--------------------+------+------+---+
only showing top 5 rows

+---------+---+----+------------------+----------+----------+-----

Conversão de tipos.

In [61]:
df_.withColumn('Height', col('Height').cast(FloatType())).show(5)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)| 169.0|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...| 169.0|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...| 167.0|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)| 167.0|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)| 170.0|    72|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows

time: 376 ms (started: 2023-09-14 17:18:12 +00:00)
